<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ollama LLM

## Setup
First, follow the [readme](https://github.com/jmorganca/ollama) to set up and run a local Ollama instance.

When the Ollama app is running on your local machine:
- All of your local models are automatically served on localhost:11434
- Select your model when setting llm = Ollama(..., model="<model family>:<version>")
- Increase defaullt timeout (30 seconds) if needed setting Ollama(..., request_timeout=300.0)
- If you set llm = Ollama(..., model="<model family") without a version it will simply look for latest
- By default, the maximum context window for your model is used. You can manually set the `context_window` to limit memory usage.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-ollama

In [ ]:
from llama_index.llms.ollama import Ollama

In [ ]:
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [ ]:
resp = llm.complete("Who is Paul Graham?")

In [ ]:
print(resp)

Paul Graham is a British-American entrepreneur, programmer, and writer. He's a prominent figure in the technology industry, known for his insights on entrepreneurship, programming, and culture.

Here are some key facts about Paul Graham:

1. **Founder of Y Combinator**: In 2005, Graham co-founded Y Combinator (YC), a startup accelerator program that provides seed funding to early-stage companies. YC has since become one of the most successful and influential startup accelerators in the world.
2. **Successful entrepreneur**: Before starting YC, Graham had already founded several successful startups, including Viaweb (acquired by Yahoo! in 2000), which developed an online store for eBay sellers, and PCGenie (sold to Apple).
3. **Writer and essayist**: Graham has written numerous essays on topics such as entrepreneurship, programming, and technology culture, which have been published on his website, paulgraham.com. His writing often explores the intersection of business, technology, and h

#### Call `chat` with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [ ]:
print(resp)

assistant: Ye be wantin' to know me name, eh? Well, matey, I be Captain Calico Jack "Blackbeak" McCoy, the most infamous buccaneer to ever sail the Seven Seas! *adjusts eye patch*

Me ship, the "Maverick's Revenge", be a sturdy galleon with three masts and a hull black as coal. She be me home, me best mate, and me ticket to riches and adventure on the high seas!

And don't ye be forgettin' me trusty parrot sidekick, Polly! She be squawkin' out sea shanties and insults to any landlubber who gets too close to our ship. *winks*

So, what brings ye to these waters? Be ye lookin' for a bit o' treasure, or just wantin' to hear tales of me swashbucklin' exploits?


### Streaming

Using `stream_complete` endpoint 

In [ ]:
response = llm.stream_complete("Who is Paul Graham?")

In [ ]:
for r in response:
    print(r.delta, end="")

Paul Graham is a British-American programmer, writer, and entrepreneur. He's best known for co-founding the online startup accelerator Y Combinator (YC) in 2005, which has become one of the most successful and influential startup accelerators in the world.

Graham was born in 1964 in Cambridge, England. He studied philosophy at Durham University and later moved to the United States to work as a programmer. In the early 1990s, he co-founded several startups, including Viaweb (later renamed to PayPal), which was acquired by eBay in 2002 for $1.5 billion.

In 2005, Graham co-founded Y Combinator with his fellow entrepreneurs Ron Conway and Robert Targ. The accelerator's goal is to help early-stage startups succeed by providing them with funding, mentorship, and networking opportunities. Over the years, YC has invested in over 2,000 companies, including notable successes like Dropbox, Airbnb, Reddit, and Stripe.

Graham is also a prolific writer and blogger on topics related to technology,

Using `stream_chat` endpoint

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.stream_chat(messages)

In [ ]:
for r in resp:
    print(r.delta, end="")

Me hearty! Me name be Captain Cutlass "Blackheart" McCoy, the most feared and revered pirate to ever sail the Seven Seas! *adjusts bandana*

Me ship, the "Maverick's Revenge", be me home sweet home, and me crew, the "Misfits of Mayhem", be me trusty mates in plunderin' and pillagin'! We sail the seas in search of treasure, adventure, and a good swig o' grog!

So, what brings ye to these waters? Are ye lookin' fer a swashbucklin' good time, or maybe just wantin' to know how to find yer lost parrot, Polly?

## JSON Mode

Ollama also supports a JSON mode, which tries to ensure all responses are valid JSON.

This is particularly useful when trying to run tools that need to parse structured outputs.

In [ ]:
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    json_mode=True,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [ ]:
response = llm.complete(
    "Who is Paul Graham? Output as a structured JSON object."
)
print(str(response))

{ "name": "Paul Graham", 
  " occupation": ["Computer Programmer", "Entrepreneur", "Venture Capitalist"], 
  "bestKnownFor": ["Co-founder of Y Combinator (YC)", "Creator of Hacker News"], 
  "books": ["Hackers & Painters: Big Ideas from the Computer Age", "The Lean Startup"], 
  "education": ["University College London (UCL)", "Harvard University"], 
  "awards": ["PC Magazine's Programmer of the Year award"], 
  "netWorth": ["estimated to be around $500 million"], 
  "personalWebsite": ["https://paulgraham.com/"] }


## Structured Outputs

We can also attach a pyndatic class to the LLM to ensure structured outputs. This will use Ollama's builtin structured output capabilities for a given pydantic class.

In [ ]:
from llama_index.core.bridge.pydantic import BaseModel


class Song(BaseModel):
    """A song with name and artist."""

    name: str
    artist: str

In [ ]:
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

sllm = llm.as_structured_llm(Song)

In [ ]:
from llama_index.core.llms import ChatMessage

response = sllm.chat([ChatMessage(role="user", content="Name a random song!")])
print(response.message.content)

{"name":"Hey Ya!","artist":"OutKast"}


Or with async

In [ ]:
response = await sllm.achat(
    [ChatMessage(role="user", content="Name a random song!")]
)
print(response.message.content)

{"name":"Mr. Blue Sky","artist":"Electric Light Orchestra (ELO)"}


You can also stream structured outputs! Streaming a structured output is a little different than streaming a normal string. It will yield a generator of the most up to date structured object.

In [ ]:
response_gen = sllm.stream_chat(
    [ChatMessage(role="user", content="Name a random song!")]
)
for r in response_gen:
    print(r.message.content)

{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":null,"artist":null}
{"name":"","artist":null}
{"name":"Mr","artist":null}
{"name":"Mr.","artist":null}
{"name":"Mr. Blue","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":null}
{"name":"Mr. Blue Sky","artist":""}
{"name":"Mr. Blue Sky","artist":"Electric"}
{"name":"Mr. Blue Sky","artist":"Electric Light"}
{"name":"Mr. Blue Sky","artist":"Electric Light Orchestra"}
{"name":"Mr. Blue Sky","artist":"Electric Light Orchestra"}
{"name":"Mr. Blue Sky","artist":"Electric Light Orchestra"}


## Multi-Modal Support

Ollama supports multi-modal models, and the Ollama LLM class natively supports images out of the box.

This leverages the content blocks feature of the chat messages.

Here, we leverage the `llama3.2-vision` model to answer a question about an image. If you don't have this model yet, you'll want to run `ollama pull llama3.2-vision`.

In [ ]:
!wget "https://pbs.twimg.com/media/GVhGD1PXkAANfPV?format=jpg&name=4096x4096" -O ollama_image.jpg

In [ ]:
from llama_index.core.llms import ChatMessage, TextBlock, ImageBlock
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="llama3.2-vision",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

messages = [
    ChatMessage(
        role="user",
        blocks=[
            TextBlock(text="What type of animal is this?"),
            ImageBlock(path="ollama_image.jpg"),
        ],
    ),
]

resp = llm.chat(messages)
print(resp)

assistant: The image depicts a cartoon alpaca wearing VR goggles and a headset, with the Google logo displayed prominently in front of it. The alpaca is characterized by its distinctive long neck, soft wool, and a pair of VR goggles perched atop its head. It is also equipped with a headset that is connected to the goggles. The alpaca's body is depicted as a cloud, which is a common visual representation of the Google brand. The overall design of the image is playful and humorous, with the alpaca's VR goggles and headset giving it a futuristic and tech-savvy appearance.


Close enough ;) 

## Thinking

Models in Ollama support "thinking" -- the process of reasoning and reflecting on a response before returning a final answer.

Below we show how to enable thinking in Ollama models in both streaming and non-streaming modes using the `thinking` parameter and the `qwen3:8b` model.

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="qwen3:8b",
    request_timeout=360,
    thinking=True,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [ ]:
resp = llm.complete("What is 434 / 22?")

In [ ]:
print(resp.additional_kwargs["thinking"])

Okay, so I need to figure out what 434 divided by 22 is. Let me start by recalling how division works. Dividing a larger number by a smaller one can sometimes be tricky, especially when the numbers aren't multiples of each other. Let me see... Maybe I can use long division here. 

First, I should check if 22 goes into 434 evenly or if there's a remainder. Let me write it out step by step. 

Starting with the first digit of 434, which is 4. But 22 is larger than 4, so I can't divide 22 into 4. Then I take the first two digits, which is 43. Now, how many times does 22 go into 43? Let me think. 22 times 1 is 22, and 22 times 2 is 44. Oh, 44 is too big because 44 is more than 43. So 22 goes into 43 once. 

So I write 1 as the first digit of the quotient. Then, I multiply 1 by 22, which gives 22. Subtract that from 43, and I get 43 - 22 = 21. 

Now, bring down the next digit from 434, which is 4. So now, the new number is 214. Wait, no, actually, after bringing down the 4, it's 214? Let me 

In [ ]:
print(resp.text)

To solve the division $ \frac{434}{22} $, we begin by simplifying the fraction.

---

### **Step 1: Simplify the Fraction**

Both 434 and 22 are divisible by 2:

$$
\frac{434}{22} = \frac{434 \div 2}{22 \div 2} = \frac{217}{11}
$$

Now, $ \frac{217}{11} $ is in its simplest form because 11 is a prime number and does not divide 217 evenly (11 × 19 = 209, and 217 − 209 = 8).

---

### **Step 2: Convert to a Mixed Number (Optional)**

We can also express $ \frac{217}{11} $ as a mixed number:

- Divide 217 by 11: $ 217 \div 11 = 19 $ with a remainder of 8.
- Therefore, $ \frac{217}{11} = 19 \frac{8}{11} $

---

### **Step 3: Convert to Decimal (Optional)**

If we convert $ \frac{217}{11} $ to a decimal:

$$
\frac{217}{11} = 19.727272\ldots
$$

This is a repeating decimal, with the "72" repeating indefinitely. We can represent this as:

$$
19.\overline{72}
$$

---

### **Final Answer**

Since the question is open-ended and does not specify the format, the exact and preferred form is the **f

Thats a lot of thinking!

Now, let's try a streaming example to make the wait less painful:

In [ ]:
resp_gen = llm.stream_complete("What is 434 / 22?")

thinking_started = False
response_started = False

for resp in resp_gen:
    if resp.additional_kwargs.get("thinking_delta", None):
        if not thinking_started:
            print("\n\n-------- Thinking: --------\n")
            thinking_started = True
            response_started = False
        print(resp.additional_kwargs["thinking_delta"], end="", flush=True)
    if resp.delta:
        if not response_started:
            print("\n\n-------- Response: --------\n")
            response_started = True
            thinking_started = False
        print(resp.delta, end="", flush=True)



-------- Thinking: --------

Okay, so I need to figure out what 434 divided by 22 is. Let me start by recalling how division works. I know that dividing a number by another means finding out how many times the divisor fits into the dividend. In this case, 22 is the divisor, and 434 is the dividend. 

First, maybe I should try to simplify this division. Let me see if both numbers can be divided by a common factor. Let me check if 22 and 434 have any common factors. The prime factors of 22 are 2 and 11. Let me check if 434 is divisible by 2. Yes, because 434 is an even number. Dividing 434 by 2 gives me 217. So, if I divide both numerator and denominator by 2, the problem becomes 217 divided by 11. That might be easier to handle.

Now, I need to divide 217 by 11. Let me do this step by step. How many times does 11 go into 21? Well, 11 times 1 is 11, and 11 times 2 is 22, which is too much. So, 1 time. Subtract 11 from 21, which leaves 10. Bring down the next digit, which is 7, making i